Why SPY
1. Volatility is high
2. Low Fees
3. Cheaper than SPX

Considerations
1. Using different markets for predicting SPY
2. Markets used
Asian
    AORD - AUS
    HSI - HK
    Nikkei - Japan
EU
    CAC40 - France
    DAX - Germany
US
    SPY
    S&P 500
    Nasdaq
    DJI

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [ ]:
aord = pd.read_csv('ALLOrdinary.csv')
nikkei = pd.read_csv('Nikkei225.csv').ffill() 
# Forward fill to handle missing data by filling with the last available value
hsi = pd.read_csv('HSI.csv')
daxi= pd.read_csv('DAXI.csv')
cac40 = pd.read_csv('CAC40.csv')
sp500 = pd.read_csv('SP500.csv')
nasdaq = pd.read_csv('nasdaq_composite.csv')
spy = pd.read_csv('SPY.csv')
dji = pd.read_csv('DJI.csv')

# Adj Close is the adjusted closing price that has been amended to include any disputions or corporate actions

$$ y = b_0 + b_1x_1 + b_2x_2 + b_3x_3 + ...$$

Response Variable: Open Next Day - Open (Tmr's Open - Tdy's Open)
Predictors:
US: Open - Open Price of Lasr Day
EU: Open - Open Last Dat (Price at noon - minus open price -> For intraday Data)
Asian: Close - Open